In [1]:
import numpy as np, os, sys, pandas as pd, csv, copy
from pathlib import Path

root = '../data'
data_info = np.array(pd.read_table(root +'/list_eval_partition.txt', header=1, delim_whitespace=True))[:,:]
#  np.array(pd.read_table(root +'/train_data/label.txt', delim_whitespace=True))[:,:]

df = pd.read_table(root +'/list_eval_partition.txt', header=1, delim_whitespace=True)

FileNotFoundError: [Errno 2] File ../data/list_eval_partition.txt does not exist: '../data/list_eval_partition.txt'

/tmp/5a5abe79-b34f-44c1-b031-ee61e629a6d9


In [27]:
df.head()

,image_name,item_id,evaluation_status
0,img/WOMEN/Dresses/id_00000002/02_1_front.jpg,id_00000002,train
1,img/WOMEN/Dresses/id_00000002/02_2_side.jpg,id_00000002,train
2,img/WOMEN/Dresses/id_00000002/02_4_full.jpg,id_00000002,train
3,img/WOMEN/Dresses/id_00000002/02_7_additional.jpg,id_00000002,train
4,img/WOMEN/Skirts/id_00000003/02_1_front.jpg,id_00000003,train


In [73]:
def get_files_from_path(pathstring):
    """retrives file names from the folder and returns a pandas dataframe with
    four columns: path, filesize, lat, long

    Arguments:
        pathstring {string} -- relative location of file

    Returns:
        [pandas dataframe] -- sorted by the filesize
    """

    filenames = []
    for file in Path(pathstring).glob("**/*.jpg"):
        
        if file.parts[-2] == "gallery" or file.parts[-2] == "query":
            evaluation_status = file.parts[-2]
        else:
            evaluation_status = "train"

        filenames.append((str(file), file.parts[2], evaluation_status))
    
    files_df = pd.DataFrame(list(filenames),
                            columns=["image_name", "item_id", "evaluation_status"])
    
    sorted_files = files_df.sort_values("item_id")
    result_df = sorted_files.reset_index(drop=True)
    return result_df

In [74]:
data = get_files_from_path("data")

In [75]:
train, query, gallery = data_info[data_info[:,2]=='train'][:,:2], data_info[data_info[:,2]=='query'][:,:2], data_info[data_info[:,2]=='gallery'][:,:2]


In [76]:
lab_conv = {x:i for i,x in enumerate(np.unique(np.array([int(x.split('_')[-1]) for x in train[:,1]])))}

train[:,1] = np.array([lab_conv[int(x.split('_')[-1])] for x in train[:,1]])

In [77]:
train = np.array(pd.read_table(root +'/train_data/label.txt', delimiter=','))[:,:]

In [78]:

train_ys, train_im_paths = [], []
for img_path, key in train:
    train_im_paths.append(os.path.join(root, 'Img', img_path))
    train_ys += [int(key)]


In [127]:
from dataset.huawei import Huawei_Dataset
from dataset.utils import make_transform
from dataset import sampler
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate
from models.resnet import *

import torch, math, time, argparse, os
import random, dataset, utils, losses, models
import numpy as np
from tqdm import *

In [128]:
# data_root = os.getcwd()
# data_root
# %cd code

[Errno 2] No such file or directory: 'code'
/media/stevel/files/computerVision/proxy_anchor_DIGIX/code


In [129]:
model = 'resnet18'
os.chdir('../data/')
data_root = os.getcwd()
trn_dataset = Huawei_Dataset(
            root = data_root,
            mode = 'train',
            transform = make_transform(
                is_train = True, 
                is_inception = (model == 'bn_inception')
            ))

In [141]:
IPC = 0
sz_batch = 150
nb_workers = 4

if IPC:
    balanced_sampler = sampler.BalancedSampler(trn_dataset, batch_size=sz_batch, images_per_class = IPC)
    batch_sampler = BatchSampler(balanced_sampler, batch_size = sz_batch, drop_last = True)
    dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        num_workers = nb_workers,
        pin_memory = True,
        batch_sampler = batch_sampler
    )
    print('Balanced Sampling')
    
else:
    dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        batch_size = sz_batch,
        shuffle = True,
        num_workers = nb_workers,
        drop_last = True,
        pin_memory = True
    )
    print('Random Sampling')

Random Sampling


# Eval Dataset HERE

In [ ]:
nb_classes = trn_dataset.nb_classes()
sz_embedding = 512
l2_norm = 1
bn_freeze = 1
loss = "Proxy_Anchor"
gpu_id = 0 

# Backbone Model
# if args.model.find('googlenet')+1:
#     model = googlenet(embedding_size=args.sz_embedding, pretrained=True, is_norm=args.l2_norm, bn_freeze = args.bn_freeze)
# elif args.model.find('bn_inception')+1:
#     model = bn_inception(embedding_size=args.sz_embedding, pretrained=True, is_norm=args.l2_norm, bn_freeze = args.bn_freeze)
if  model.find('resnet18')+1:
    model = Resnet18(embedding_size=sz_embedding, pretrained=True, is_norm=l2_norm, bn_freeze = bn_freeze)
# elif model.find('resnet50')+1:
#     model = Resnet50(embedding_size=sz_embedding, pretrained=True, is_norm=l2_norm, bn_freeze = bn_freeze)
# elif model.find('resnet101')+1:
#     model = Resnet101(embedding_size=sz_embedding, pretrained=True, is_norm=l2_norm, bn_freeze = bn_freeze)
print(model)
model = model.cuda()

if gpu_id == -1:
    model = nn.DataParallel(model)

In [ ]:
mrg = 0.1
alpha = 32
lr = 1e-4

if loss == 'Proxy_Anchor':
    criterion = losses.Proxy_Anchor(nb_classes = nb_classes, sz_embed = sz_embedding, mrg = mrg, alpha = alpha).cuda()
elif loss == 'Proxy_NCA':
    criterion = losses.Proxy_NCA().cuda()
elif loss == 'MS':
    criterion = losses.MultiSimilarityLoss().cuda()
elif loss == 'Contrastive':
    criterion = losses.ContrastiveLoss().cuda()
elif loss == 'Triplet':
    criterion = losses.TripletLoss().cuda()
elif loss == 'NPair':
    criterion = losses.NPairLoss().cuda()

In [ ]:
# Train Parameters
param_groups = [
    {'params': list(set(model.parameters()).difference(set(model.model.embedding.parameters()))) if gpu_id != -1 else 
                 list(set(model.module.parameters()).difference(set(model.module.model.embedding.parameters())))},
    {'params': model.model.embedding.parameters() if gpu_id != -1 else model.module.model.embedding.parameters(), 'lr':float(lr) * 1},
]
if loss == 'Proxy_Anchor':
    param_groups.append({'params': criterion.proxies, 'lr':float(lr) * 100})


In [ ]:
optimizer = "adamw"
weight_decay = 1e-4
lr_decay_step = 10
lr_decay_gamma = 0.5


# Optimizer Setting
if optimizer == 'sgd': 
    opt = torch.optim.SGD(param_groups, lr=float(lr), weight_decay = weight_decay, momentum = 0.9, nesterov=True)
elif optimizer == 'adam': 
    opt = torch.optim.Adam(param_groups, lr=float(lr), weight_decay = weight_decay)
elif optimizer == 'rmsprop':
    opt = torch.optim.RMSprop(param_groups, lr=float(lr), alpha=0.9, weight_decay = weight_decay, momentum = 0.9)
elif optimizer == 'adamw':
    opt = torch.optim.AdamW(param_groups, lr=float(lr), weight_decay = weight_decay)
    
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=lr_decay_step, gamma = lr_decay_gamma)

In [138]:
nb_epochs = 10
warm = 1
# print("Training parameters: {}".format(vars(args)))
print("Training for {} epochs.".format(nb_epochs))
losses_list = []
best_recall=[0]
best_epoch = 0

for epoch in range(0, nb_epochs):
    model.train()
    bn_freeze = bn_freeze
    if bn_freeze:
        modules = model.model.modules() if gpu_id != -1 else model.module.model.modules()
        for m in modules: 
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    losses_per_epoch = []
    
    # Warmup: Train only new params, helps stabilize learning.
    if warm > 0:
        if gpu_id != -1:
            unfreeze_model_param = list(model.model.embedding.parameters()) + list(criterion.parameters())
        else:
            unfreeze_model_param = list(model.module.model.embedding.parameters()) + list(criterion.parameters())

        if epoch == 0:
            for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
                param.requires_grad = False
        if epoch == warm:
            for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
                param.requires_grad = True

    pbar = tqdm(enumerate(dl_tr))

    for batch_idx, (x, y) in pbar:                         
        m = model(x.squeeze().cuda())
        loss = criterion(m, y.squeeze().cuda())
        
        opt.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)
        if args.loss == 'Proxy_Anchor':
            torch.nn.utils.clip_grad_value_(criterion.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(dl_tr),
                100. * batch_idx / len(dl_tr),
                loss.item()))
        
    losses_list.append(np.mean(losses_per_epoch))
    # wandb.log({'loss': losses_list[-1]}, step=epoch)
    scheduler.step()

Training for 10 epochs.


ValueError: 'a' cannot be empty unless no samples are taken

ValueError: 'a' cannot be empty unless no samples are taken